In [1]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
import os

In [3]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))


vgg_dir = os.path.join(BASE_DIR, "models", "VGG19_trained.keras")
nn_dir = os.path.join(BASE_DIR, "models", "NN_trained.keras")
val_dir = os.path.join(BASE_DIR, "data", "preprocessed_split_2","val")


vgg_model = tf.keras.models.load_model(vgg_dir)
nn_model = tf.keras.models.load_model(nn_dir)

In [4]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 16

val_gen_nn = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False,
    color_mode="grayscale"
)


val_gen_vgg = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False,
    color_mode="rgb"
)

Found 661 images belonging to 2 classes.
Found 661 images belonging to 2 classes.


In [5]:
def get_binary_metrics(model, generator):
    y_true = generator.classes
    y_pred_probs = model.predict(generator, verbose=0)
    
    # Convert probabilities to class labels
    if y_pred_probs.ndim > 1 and y_pred_probs.shape[1] == 1:
        y_pred = (y_pred_probs > 0.5).astype(int).flatten()
    else:
        y_pred = np.argmax(y_pred_probs, axis=1)
    
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    return {"Accuracy": acc, "Precision": precision, "Recall": recall, "F1-Score": f1}

In [6]:
vgg_metrics = get_binary_metrics(vgg_model, val_gen_vgg)
nn_metrics = get_binary_metrics(nn_model, val_gen_nn)


print("Metric        | VGG19      | NN")
print("--------------------------------------")
for key in vgg_metrics.keys():
    print(f"{key:13} | {vgg_metrics[key]:.4f}   | {nn_metrics[key]:.4f}")

Metric        | VGG19      | NN
--------------------------------------
Accuracy      | 0.6520   | 0.6309
Precision     | 0.5978   | 0.5858
Recall        | 0.9763   | 0.9497
F1-Score      | 0.7416   | 0.7246
